<a href="https://colab.research.google.com/github/jinocryst/data-analysis-with-pandas/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# 오픈소스 기반 데이터 분석 - 중간과제물


### 1-2 Python을 사용하여 API를 호출하는 코드를 작성하고, 2015년 1월부터 2024년 12월까지의 개인 유형의 현년 전기, 가스, 수도, 지역난방 에너지 사용량 데이터를 수집하는 프로그램을 작성하시오. API 호출 성공을 확인할 수 있는 실행 결과를 캡처하여 첨부하시오.

In [22]:
import requests
import json


responses = []    # API Request json 수집용 리스트

def get_year_months():
  year_months = []
  for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')

  #print("생성된 연도-월 목록:")
  #print(year_months)
  #print(f"총 {len(year_months)}개 (2015.01 ~ 2024.12)")
  return year_months

def get_responses():

  # 기간중의 모든 응답결과 중 list_total_count 체크한 결과 7이였으므로 , 5보다 큰 10으로 변경하였음
  base_url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataSummaryInfo/1/10'

  success_count = 0 # API Request 성공건수
  fail_count = 0    # API Request 실패건수

  for year_month in get_year_months():
    url = f'{base_url}/{year_month[:4]}/{year_month[4:]}'

    try:
      response = requests.get(url)
      response.raise_for_status()
      data = response.json()

      responses.append(data)
      # print(f"{year_month} 데이터 수집 완료")

    except requests.exceptions.RequestException as e:
      print(f"{year_month} API 호출 실패: {e}")
    except json.JSONDecodeError as e:
      print(f"{year_month} JSON 파싱 실패: {e}")

    if response.status_code == 200:
      # print("api 호출 성공")
      # print(response.json())
      success_count += 1
    else:
      # print(f"API 호출 실패: {response.status_code}")
      fail_count += 1

  # 결과 확인용
  if responses:

      # 전체 결과들의 list_total_count 값 확인하여 파라미터변경 필요여부 확인코드(모든 결과가 7건씩이였음)
      # total_counts = [
      # r.get('energyUseDataSummaryInfo', {}).get('list_total_count', None)
      #   for r in responses
      # ]
      # print("list_total_count 목록:", total_counts)
    print(f"총 요청 수: {len(responses)}건")
    print(f"성공 건수: {success_count}건")
    print(f"실패 건수: {fail_count}건")

    return responses


def main():

    personal_data = []
    responses = get_responses()
    for r in responses:
      info = r.get("energyUseDataSummaryInfo", {})
      result = info.get("RESULT", {})
      code = result.get("CODE", "UNKNOWN")

      if code == "INFO-000":  # 정상인경우
        rows = info.get("row", [])
        for row in rows:
          # 개인 유형만 추출
          if row.get("MM_TYPE") == "개인":
              personal_data.append({
                  "YEAR": row.get("YEAR"),
                  "MON": row.get("MON"),
                  "EUS": row.get("EUS"),  #전기사용량(EUS)
                  "GUS": row.get("GUS"),  #가스사용량(GUS)
                  "WUS": row.get("WUS"),  #수도사용량(WUS)
                  "HUS": row.get("HUS"),  #지역난방(HUS)
              })
      else:
        print

    # 결과 출력
    print(f"‘개인’ 유형 데이터 수집 성공 건수: {len(personal_data)}건\n") # 기대치 120건
    for i, data in enumerate(personal_data, start=1):
      print(f"[{i}] [{data['YEAR']}년 {data['MON']}월] 전기사용량(EUS): {data['EUS']} , 가스사용량(GUS): {data['GUS']} , 수도사용량(WUS): {data['WUS']} , 지역난방(HUS): {data['HUS']}")


if __name__ == "__main__":
    main()

총 요청 수: 120건
성공 건수: 120건
실패 건수: 0건
‘개인’ 유형 데이터 수집 성공 건수: 120건

[1] [2015년 01월] 전기사용량(EUS): 193784708 , 가스사용량(GUS): 59133720 , 수도사용량(WUS): 12819757.886 , 지역난방(HUS): 22740838.937
[2] [2015년 02월] 전기사용량(EUS): 189974230 , 가스사용량(GUS): 56487358 , 수도사용량(WUS): 12656888.218 , 지역난방(HUS): 18793320.213
[3] [2015년 03월] 전기사용량(EUS): 175215719 , 가스사용량(GUS): 49984873 , 수도사용량(WUS): 12655474.249 , 지역난방(HUS): 13826348.508
[4] [2015년 04월] 전기사용량(EUS): 183891769 , 가스사용량(GUS): 37306841 , 수도사용량(WUS): 12954773.246 , 지역난방(HUS): 5836002.885
[5] [2015년 05월] 전기사용량(EUS): 177289769 , 가스사용량(GUS): 22060892 , 수도사용량(WUS): 13086734.214 , 지역난방(HUS): 1320895.801
[6] [2015년 06월] 전기사용량(EUS): 186073153 , 가스사용량(GUS): 12677645 , 수도사용량(WUS): 14241651.402 , 지역난방(HUS): 294312.634
[7] [2015년 07월] 전기사용량(EUS): 165114568 , 가스사용량(GUS): 8893640 , 수도사용량(WUS): 12083879.038 , 지역난방(HUS): 702863.645
[8] [2015년 08월] 전기사용량(EUS): 242000479 , 가스사용량(GUS): 7619018 , 수도사용량(WUS): 15164553.742 , 지역난방(HUS): 460288.965
[9] [2015년 09월] 전기사용량(EUS): 2075408

### 2-1 수집한 JSON 형태의 데이터를 pandas DataFrame으로 변환하고, 데이터의 기본 정보를 출력하는 코드와 실행 결과를 첨부하시오.



In [31]:
import requests
import json
import pandas as pd


responses = []    # API Request json 수집용 리스트

def get_year_months():
  year_months = []
  for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')

  #print("생성된 연도-월 목록:")
  #print(year_months)
  #print(f"총 {len(year_months)}개 (2015.01 ~ 2024.12)")
  return year_months

def get_responses():

  # 기간중의 모든 응답결과 중 list_total_count 체크한 결과 7이였으므로 , 5보다 큰 10으로 변경하였음
  base_url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataSummaryInfo/1/10'

  success_count = 0 # API Request 성공건수
  fail_count = 0    # API Request 실패건수

  for year_month in get_year_months():
    url = f'{base_url}/{year_month[:4]}/{year_month[4:]}'

    try:
      response = requests.get(url)
      response.raise_for_status()
      data = response.json()

      responses.append(data)
      # print(f"{year_month} 데이터 수집 완료")

    except requests.exceptions.RequestException as e:
      print(f"{year_month} API 호출 실패: {e}")
    except json.JSONDecodeError as e:
      print(f"{year_month} JSON 파싱 실패: {e}")

    if response.status_code == 200:
      # print("api 호출 성공")
      # print(response.json())
      success_count += 1
    else:
      # print(f"API 호출 실패: {response.status_code}")
      fail_count += 1

  # 결과 확인용
  if responses:

      # 전체 결과들의 list_total_count 값 확인하여 파라미터변경 필요여부 확인코드(모든 결과가 7건씩이였음)
      # total_counts = [
      # r.get('energyUseDataSummaryInfo', {}).get('list_total_count', None)
      #   for r in responses
      # ]
      # print("list_total_count 목록:", total_counts)
    print(f"총 요청 수: {len(responses)}건")
    print(f"성공 건수: {success_count}건")
    print(f"실패 건수: {fail_count}건")

    return responses


def main():

    personal_data = []
    # responses = get_responses()
    with open('APIdata.json', 'r', encoding='utf-8') as f:
      responses = json.load(f)
    for r in responses:
      info = r.get("energyUseDataSummaryInfo", {})
      result = info.get("RESULT", {})
      code = result.get("CODE", "UNKNOWN")

      if code == "INFO-000":  # 정상인경우
        rows = info.get("row", [])
        for row in rows:
          # 개인 유형만 추출
          if row.get("MM_TYPE") == "개인":
              personal_data.append({
                  "YEAR": row.get("YEAR"),
                  "MON": row.get("MON"),
                  "EUS": row.get("EUS"),  #전기사용량(EUS)
                  "GUS": row.get("GUS"),  #가스사용량(GUS)
                  "WUS": row.get("WUS"),  #수도사용량(WUS)
                  "HUS": row.get("HUS"),  #지역난방(HUS)
              })
      else:
        print

    # 결과 출력
    print(f"‘개인’ 유형 데이터 수집 성공 건수: {len(personal_data)}건\n") # 기대치 120건

    # DataFrame 변환
    df = pd.DataFrame(personal_data)
    print(df)

if __name__ == "__main__":
    main()

‘개인’ 유형 데이터 수집 성공 건수: 120건

     YEAR MON        EUS       GUS           WUS           HUS
0    2015  01  193784708  59133720  12819757.886  22740838.937
1    2015  02  189974230  56487358  12656888.218  18793320.213
2    2015  03  175215719  49984873  12655474.249  13826348.508
3    2015  04  183891769  37306841  12954773.246   5836002.885
4    2015  05  177289769  22060892  13086734.214   1320895.801
..    ...  ..        ...       ...           ...           ...
115  2024  08  463971328   7072251    15873544.6    130082.263
116  2024  09  409872908   5729131    15143321.6   3420507.136
117  2024  10  282268119   8233421    15297669.9   2695491.396
118  2024  11  257927956  17418221    14678672.5   8836296.454
119  2024  12  134828126  42103424     7326853.6   1513199.277

[120 rows x 6 columns]


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [ ]:
import re
## 정규식 라이브러리

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f :
  f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [ ]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.031948089599609375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-19T06:15', 'interval': 900, 'temperature_2m': 19.0}}
서울시 종로구의 현재 온도는 : 19.0°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0   268M      0 --:--:-- --:--:-- --:--:--  268M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 448 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 mesa-vulkan-drivers amd64 23.2.1-1ubuntu3.1~22.04.3 [10.7

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)

## 사이트 접속 대기
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)

## 드라이버 종료
driver.quit()

['\n\t\tAlert \n\t\t\n\t']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정
url = 'https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m'
api_key = ''

# params = {
#     'serviceKey': api_key,
#     'returnType': 'json',
#     'numOfRows': '100',
#     'pageNo': '1',
#     'sidoName': '서울',
#     'ver': '1.0'
# }

params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}


## 데이터 수집
response = requests.get(url, params=params)
response.raise_for_status()
## 호출 성공/실패 출력
print(response.status_code)
print(response.json())


200
{'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.02491474151611328, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-19T06:30', 'interval': 900, 'temperature_2m': 18.5}}
